## Cyclistic : How Does a Bike-Share Navigate Speedy Success?

### Data source: 
[Motivate International Inc.](https://divvy-tripdata.s3.amazonaws.com/index.html)

### Initial R code source: 
This analysis is for case study 1 from the Google Data Analytics Certificate (Cyclistic).

In [ ]:
## ----Install necessary packages---------------------------------------------
options=(warn=-1)
install.packages("tidyverse")
install.packages("modeest")

## ----Load necessary packages------------------------------------------------
library(tidyverse)
library(dplyr)
library(ggplot2)
library(lubridate)
library(readxl)
library(modeest)

## Ask

#### Identify the business task.
The key business task in this case is to discover how casual riders and Cyclistic members use their rental bikes differently.  Both the Director of Marketing as well as finance analysts have concluded that annual members are more profitable.  

Therefore, the results of this analysis will be used to design a new marketing strategy to convert casual riders to annual members. 

#### Consider key stakeholders.
Key stakeholders include:  Cyclistic executive team, Director of Marketing (Lily Moreno), Marketing Analytics team.  

## Prepare

#### Download data and store it appropriately.
Data has been downloaded from [Motivate International Inc.](https://divvy-tripdata.s3.amazonaws.com/index.html)  Local copies have been stored securely on Google Drive and here on Kaggle.

#### Identify how it’s organized.
All trip data is in comma-delimited (.CSV) format with 15 columns, including:  ride ID #, ride type, start/end time, ride length (in minutes), day of the week, starting point (code, name, and latitude/longitude), ending point (code, name, and latitude/longitude), and member/casual rider.  

#### Determine the credibility of the data.
Due to the fact that this is a case study using public data, we are going to assume the data is credible.  

## Process

#### Check the data for errors.
The code chunk below will import 12 individual .xlsx files as data frames, each representing 1 of the last 12 months of trip data.  Some parsing errors persist, however, they represent less 0.25% of the data set, so this is still a representative sample.  

In [ ]:
## ----Gather data------------------------------------------------------------
options(warn=-1)
y2020_04 <- read_xlsx("../input/cyclistic-case-study/2020_04.xlsx")
y2020_05 <- read_xlsx("../input/cyclistic-case-study/2020_05.xlsx")
y2020_06 <- read_xlsx("../input/cyclistic-case-study/2020_06.xlsx")
y2020_07 <- read_xlsx("../input/cyclistic-case-study/2020_07.xlsx")
y2020_08 <- read_xlsx("../input/cyclistic-case-study/2020_08.xlsx")
y2020_09 <- read_xlsx("../input/cyclistic-case-study/2020_09.xlsx")
y2020_10 <- read_xlsx("../input/cyclistic-case-study/2020_10.xlsx")
y2020_11 <- read_xlsx("../input/cyclistic-case-study/2020_11.xlsx")
y2020_12 <- read_xlsx("../input/cyclistic-case-study/2020_12.xlsx")
y2021_01 <- read_xlsx("../input/cyclistic-case-study/2021_01.xlsx")
y2021_02 <- read_xlsx("../input/cyclistic-case-study/2021_02.xlsx")
y2021_03 <- read_xlsx("../input/cyclistic-case-study/2021_03.xlsx")

#### Choose your tools.
For this analysis, we will be using R for it's easy statistical analysis tools and data visualizations.  

#### Transform the data so you can work with it effectively.
The code chunk below will transform the data to fix data type inconsistencies found during the import process so the data can be combined into one large data frame.  

In [ ]:
## ----Data type inconsistencies exist start/end_station_id(12,01,02,03)------
options=(warn=-1)
y2020_12 <- mutate(y2020_12, start_station_id = as.numeric(start_station_id),
                   end_station_id = as.numeric(end_station_id))
y2021_01 <- mutate(y2021_01, start_station_id = as.numeric(start_station_id),
                   end_station_id = as.numeric(end_station_id))
y2021_02 <- mutate(y2021_02, start_station_id = as.numeric(start_station_id),
                   end_station_id = as.numeric(end_station_id))
y2021_03 <- mutate(y2021_03, start_station_id = as.numeric(start_station_id),
                   end_station_id = as.numeric(end_station_id))

#### Document the cleaning process.
Prior to import into R, 1 column was added to each of the 12 monthly .CSV files:  
* Day of the week (1 = Sunday, 7 = Saturday)

The .CSV files were then saved as Excel (.xlsx) files.  

The start_station_id and end_station_id fields in 4 of the .xlsx files imported as the 'character' data type when they should have been numbers (y2020_12, y2021_01, y2021_02, y2021_03).  These fields were changed to numbers using the mutate() function above.

## Analyze

#### Aggregate your data so it’s useful and accessible.
This code chunk will combine the 12 individual data frames into one large data frame for analysis.  

In [ ]:
## ----Merge individual monthly data frames into one large data frame---------
all_trips_init <- bind_rows(
                       y2020_04, 
                       y2020_05, 
                       y2020_06, 
                       y2020_07, 
                       y2020_08, 
                       y2020_09, 
                       y2020_10, 
                       y2020_11, 
                       y2020_12, 
                       y2021_01, 
                       y2021_02, 
                       y2021_03, 
                       )

Next, we add columns to list the date, month, day, and year of each ride for additional aggregation capabilities.

In [ ]:
## ----Add columns that list the date, month, day, and year of each ride for additional aggregation----
all_trips_init$date <- as.Date(all_trips_init$started_at)
all_trips_init$month <- format(as.Date(all_trips_init$date), "%m")
all_trips_init$day <- format(as.Date(all_trips_init$date), "%d")
all_trips_init$year <- format(as.Date(all_trips_init$date), "%Y")
all_trips_init$day_of_week <- format(as.Date(all_trips_init$date), "%A")

We also remove any unnecessary columns (erroneous ride_length, latitude/longitude fields).

In [ ]:
## ----Remove unnecessary columns---------------------------------------------
all_trips_init <- all_trips_init %>% 
  select(-c(ride_length, start_lat, start_lng, end_lat, end_lng))

We'll add another column to calculate each ride length in seconds.  

In [ ]:
## ---------------------------------------------------------------------------
all_trips_init$ride_length <- as.numeric(difftime(all_trips_init$ended_at,all_trips_init$started_at))

Let's next remove the NA rows.

In [ ]:
## ---------------------------------------------------------------------------
all_trips_no_na <- drop_na(all_trips_init)

The aggregated data frame includes approximately 10,500 entries (0.30% of total rides) when bikes were taken out of docks and checked for quality by Cyclistic or ride_length was negative.  We will create a new version of the data frame since data is being removed.

In [ ]:
## ----Remove negative ride length and quality check rows---------------------
all_trips <- all_trips_no_na[!(all_trips_no_na$start_station_name == "HQ QR" | all_trips_no_na$ride_length<0),]

#### Perform calculations.

Let's first look at a statistical summary of the aggregated and transformed data frame.  Let's also look at the structure of the columns. 

In [ ]:
## ----stats------------------------------------------------------------------
summary(all_trips)
str(all_trips)

Let's next focus on the average length of each ride, this time in minutes.  We see that on average, each ride is close to 30 minutes.  We'll then break that down by casual riders versus members.

In [ ]:
## ----ride_length calculations-----------------------------------------------
summary(all_trips$ride_length)/60

Looking at casual riders versus members, we can see that the average casual ride is about 46 minutes compared to the members' average ride of 16 minutes.  The median rides are 22 minutes and 12 minutes respectively.

In [ ]:
## ----Members vs. Casual Riders----------------------------------------------
aggregate(all_trips$ride_length/60 ~ all_trips$member_casual, FUN = mean)
aggregate(all_trips$ride_length/60 ~ all_trips$member_casual, FUN = median)

Using the statistical mode, we see that the most common day for renting bikes is Saturday.

In [ ]:
## ----Mode of data-----------------------------------------------------------
aggregate(all_trips$day_of_week ~ all_trips$member_casual, FUN = mfv)

Also, we can take a look at the average ride time by day for members and casual riders with duration again in minutes.  Regardless of day of the week, casual users ride 2.7x to 3x  longer than members, with both groups riding longer on weekends.

In [ ]:
## ---------------------------------------------------------------------------
all_trips$day_of_week <- ordered(all_trips$day_of_week, levels=c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"))
aggregate(all_trips$ride_length/60 ~ all_trips$member_casual + all_trips$day_of_week, FUN = mean)

In [ ]:
## ---------------------------------------------------------------------------
options(warning=-1)
all_trips %>% 
  mutate(weekday = wday(started_at, label = TRUE)) %>%
  group_by(member_casual, weekday) %>%  
  summarise(number_of_rides = n()  
  ,average_duration = mean(ride_length/60)) %>% 		
  arrange(member_casual, weekday)	

#### Identify trends and relationships.
Here are some key observations using the simple analyses above:  
* Casual riders average longer rides than members.  This makes sense as members are likely using the bike rental service for particular commutes (i.e. work or school) whereas casual riders are just that--casual (perhaps they are sightseeing).   
* The number of rides for both types of users starts off slow on Mondays, gradually increases to a peak on Saturdays with a small drop off on Sundays.  

## Share

#### Determine the best way to share your findings.

Certain simple findings work well as small tables as shown above.  Other findings we will present using ggplot2 for data visualizations.  

#### Create effective data visualizations.

In [ ]:
## ----Number of rides by rider type------------------------------------------
all_trips %>% 
  mutate(weekday = wday(started_at, label = TRUE)) %>% 
  group_by(member_casual, weekday) %>% 
  summarise(number_of_rides = n()
            ,average_duration = mean(ride_length)) %>% 
  arrange(member_casual, weekday)  %>% 
  ggplot(aes(x = weekday, y = number_of_rides, fill = member_casual)) +
  geom_col(position = "dodge") + 
  labs(title = "Table 1: Number of Rides by Day and Rider Type") + 
  ylab("Number of Rides (1e+05 = 100,000)") + 
  xlab("Day of Week")

## ----Average duration-------------------------------------------------------
all_trips %>% 
  mutate(weekday = wday(started_at, label = TRUE)) %>% 
  group_by(member_casual, weekday) %>% 
  summarise(number_of_rides = n()
            ,average_duration = mean(ride_length/60)) %>% 
  arrange(member_casual, weekday)  %>% 
  ggplot(aes(x = weekday, y = average_duration, fill = member_casual)) +
  geom_col(position = "dodge") + 
  labs(title = "Table 2: Average Ride Duration by Day and Rider Type") + 
  ylab("Average Duration (minutes)") + 
  xlab("Day of Week")

## ----Number of rides by bike type and rider type----------------------------
all_trips %>% 
  group_by(member_casual, rideable_type) %>% 
  summarise(number_of_rides = n()
            ,average_duration = mean(ride_length)) %>% 
  arrange(member_casual, rideable_type)  %>% 
  ggplot(aes(x = rideable_type, y = number_of_rides, fill = member_casual)) +
  geom_col(position = "dodge") + 
  labs(title = "Table 3: Number of Rides by Bike Type and Rider Type") + 
  ylab("Number of Rides (5e+05 = 500,000)") + 
  xlab("Bike Type")

## ----Number of rides by day and bike type-----------------------------------
all_trips %>% 
  mutate(weekday = wday(started_at, label = TRUE)) %>% 
  group_by(rideable_type, weekday) %>% 
  summarise(number_of_rides = n()
            ,average_duration = mean(ride_length)) %>% 
  arrange(rideable_type, weekday)  %>% 
  ggplot(aes(x = weekday, y = number_of_rides, fill = rideable_type)) +
  geom_col(position = "dodge") + 
  labs(title = "Table 4: Number of Rides by Day and Bike Type") + 
  ylab("Number of Rides (1e+05 = 100,000)") + 
  xlab("Day of Week")

## ----Average duration by bike type------------------------------------------
all_trips %>% 
  mutate(weekday = wday(started_at, label = TRUE)) %>% 
  group_by(rideable_type, weekday) %>% 
  summarise(number_of_rides = n()
            ,average_duration = mean(ride_length/60)) %>% 
  arrange(rideable_type, weekday)  %>% 
  ggplot(aes(x = weekday, y = average_duration, fill = rideable_type)) +
  geom_col(position = "dodge") + 
  labs(title = "Table 5: Average Ride Duration by Day and Bike Type") + 
  ylab("Average Duration (minutes)") + 
  xlab("Day of Week")

## ----Number of rides by month and rider type--------------------------------
all_trips %>% 
  group_by(member_casual, month) %>% 
  summarise(number_of_rides = n()
            ,average_duration = mean(ride_length)) %>% 
  arrange(member_casual, month)  %>% 
  ggplot(aes(x = month, y = number_of_rides, group = member_casual)) +
  geom_line(aes(color = member_casual)) + 
  geom_point() +
  labs(title = "Table 6: Number of Rides by Month and Rider Type") + 
  ylab("Number of Rides (1e+05 = 100,000)") + 
  xlab("Month")

#### Present your findings.

Here is a summary of the key observations from above:  
* On average, each ride is about 30 minutes:  
    + Casual users ride for 46 minutes on average.  
    + Members ride for 16 minutes on average.  
* Regardless of being a member or not, the most popular day to rent a bike is Saturday.  
* Bike rentals start off at a low on Mondays, peak on Saturdays with a slight drop off on Sundays (Table 1).  
* Members rent bikes on a more consistent basis throughout the entire week, whereas casual rentals are low Monday through Thursday and peak towards the weekend (Table 1).  
* On any day of the week, casual users ride 2.7x to 3x longer than members (Table 2).  
* The docked bike option is far more popular than both classic bikes and electric bikes, both in terms of number of rentals (Table 3, Table 4) and average ride duration on each type of bike (Table 5).  
* Bike rentals follow a seasonal pattern for both types of users (Table 6).  Since Chicago experiences inclement weather, lowest usage is in the winter with rentals starting to ramp up in the spring.  Peak usage is in the summer (August) before it starts to decline again during the Fall.  

## Act

#### Your top three recommendations based on your analysis

First, let's revisit the key business task:  *discover how casual riders and Cyclistic members use their rental bikes differently.*   

It appears that members rent bikes for regular commuting---perhaps to work or school.  This is based on the observation that the number of rides is fairly consistent day over day (Table 1) as is the average ride duration (Table 2).  

Conversely, casual user rentals are lower Monday through Thursday and trend upwards starting on Friday and peaking on the weekend (Table 1).  This seems to indicate using the bikes for leisure activities such as touring the city, sightseeing, etc.  Additional data that supports this is the longer average ride duration (Table 2).  

My three recommendations for the new marketing strategy are as follows:  
1. Offer a weekend-only membership at a different price point than the full annual membership to entice casual users towards a full annual membership.  They can only unlock bikes on Friday, Saturday, or Sunday.  
2. Create a "See our City" campaign targeted to casual users that includes 52 suggested routes that will cover all of the major sights in Chicago---one for each weekend.  Casual riders *could* rent as they go, but they could see their city in one year (and save money!) for the price of an annual membership.  
3. Be sure that any campaigns are out to the market in the summer when ridership is at its annual peak.